In [1]:
import ee
import geemap
import datetime
import geopandas as gpd
import os
import zipfile

In [2]:
# Authenticate google earth engine
ee.Authenticate()

In [ ]:
# Initialize Earth Engine
ee.Initialize(project='Specify your project name from Google Earth Engine e.g ee-myproject')

url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BGD_shp.zip"
shp_zip = "gadm_bgd.zip"


url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BGD_shp.zip"
shp_zip = "gadm_bgd.zip"
geemap.download_file(url, shp_zip)


if not os.path.exists("gadm_bgd"):
    with zipfile.ZipFile(shp_zip, 'r') as zip_ref:
        zip_ref.extractall("gadm_bgd")


gdf = gpd.read_file("gadm_bgd/gadm41_BGD_2.shp")

# Get Dhaka district
dhaka_gdf = gdf[gdf["NAME_2"] == "Dhaka"]



Downloading...
From: https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BGD_shp.zip
To: /content/gadm_bgd.zip
100%|██████████| 111M/111M [00:02<00:00, 37.3MB/s]


Extracting files...


In [4]:
aoi = geemap.gdf_to_ee(dhaka_gdf)

#Visualise AOI
Map = geemap.Map(center=[23.81, 90.41], zoom=10)
Map.addLayer(aoi, {}, "Dhaka AOI")
Map

Map(center=[23.81, 90.41], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [92]:
# Cloud masking function used for 2016 to 2023

def maskS2clouds(image):
    qa = image.select('QA10')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)






In [5]:
# Cloud masking function used for 2024
def maskS2clouds(image):
    qa = image.select('QA60')
    cloud_bit = 1 << 10  # value 1024
    cirrus_bit = 1 << 11  # value 2048
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    return image.updateMask(mask)

In [6]:
# NDVI calculation function
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)



In [13]:
def get_monthly_ndvi(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    monthly_col = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterBounds(aoi)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))) \
                  .map(lambda img: img.clip(aoi))

    count = monthly_col.size()
    empty = ee.Algorithms.IsEqual(count, 0)

    def compute():
        processed = monthly_col.map(maskS2clouds).map(addNDVI)
        return (processed.select('NDVI').median()
                .set('year', year)
                .set('month', month)
                .set('date', start_date.format('YYYY-MM'))
                .set('empty', False))

    empty_image = ee.Image(0).rename('NDVI') \
        .set('year', year) \
        .set('month', month) \
        .set('empty', True)

    return ee.Image(ee.Algorithms.If(empty, empty_image, compute()))


In [14]:
# Batch export function
def export_ndvi(image):
    props = image.toDictionary(['year', 'month', 'empty']).getInfo()

    if props.get('empty') is True:
        print(f"⚠️ Skipping export: No data for {props['year']}-{props['month']:02d}")
        return 'skipped'

    year = props['year']
    month = props['month']

    # Extract the coordinates from the geometry for the region
    region = aoi.geometry().bounds()

    task = ee.batch.Export.image.toDrive(
        image=image.select('NDVI'),
        description=f'NDVI_{year}_{month:02d}',
        folder='GEE_Exports',
        fileNamePrefix=f'dhaka_ndvi_{year}_{month:02d}',
        scale=10,
        region=region,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        crs='EPSG:4326'
    )
    task.start()
    return task.id



In [ ]:
# Main processing
tasks = []
for year in range(2016, 2025):
    for month in range(1, 13):
        print(f'Processing {year}-{month:02d}')
        monthly_ndvi = get_monthly_ndvi(year, month)
        task_id = export_ndvi(monthly_ndvi)
        tasks.append((year, month, task_id))
        print(f'Started export: {task_id}')

# Save task IDs for reference
with open('ndvi_export_tasks.csv', 'w') as f:
    f.write('year,month,task_id\n')
    for task in tasks:
        f.write(f"{task[0]},{task[1]},{task[2]}\n")

print("All tasks submitted. Check Earth Engine Tasks tab for progress.")